In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import glob

read files

In [7]:
web_pages_dir = '../../personalisation_project/TVNZ/*/*'
files = glob.glob(web_pages_dir)

scraping

In [100]:
def fetch_meta(soup):
    image = soup.select_one('.Hero-image-inner img').attrs['src'].strip()
    content_container = soup.select_one('.Hero-container-content-inner')
    title = content_container.select_one('.Hero-title a').text.strip()
    description = content_container.select_one('.Hero-description').text.strip()

    return {
        'title': title,
        'description': description,
        'image_url': image
    }


def fetch_episodes(soup, unique_id):
    episodes = soup.select('.Episodes-list')

    data = []

    for episode in episodes:
        image_dump = episode.select_one('img').attrs['src'].strip()

        title = episode.select_one('.Episode-title-label').text.strip()
        description_container = episode.select_one('.Episode-description')
        rating = description_container.select_one('.Episode-rating').text.strip()
        description = description_container.text.strip()
        description = description[len(rating):].strip()

        data.append({
            'title':title,
            'rating': rating,
            'description': description,
            'image_dump': image_dump,
            'tv_program_id': unique_id
        })

    return data



In [105]:
%%time

tvprogram_data = []
episodes_data = []

for idx in range(len(files)):
    unique_id = idx
    file = files[idx]
    splitted = file.split('/')
    category = splitted[4]
    file_path = '/'.join(splitted[4:])

    print(file_path)

    soup = BeautifulSoup(open(file), 'html.parser')

    content = fetch_meta(soup)
    content['tv_program_id'] = unique_id
    content['category'] = category

    episodes = fetch_episodes(soup, unique_id)

    tvprogram_data.append(content)
    episodes_data += episodes

    break

documentary/proof.html
CPU times: user 37.6 ms, sys: 2.49 ms, total: 40.1 ms
Wall time: 38.5 ms


In [106]:
df_tv_programs = pd.DataFrame(tvprogram_data)
df_tv_programs.head()

,title,description,image_url,tv_program_id,category
0,Proof,"Award-winning journalist, Guyon Espiner, inves...",https://play-images-prod-ctf.tech.tvnz.co.nz/a...,0,documentary


In [ ]:
df_tv_programs.to_csv('../data/tv_programs.csv', index=False)

In [109]:
df_episodes = pd.DataFrame(episodes_data)
df_episodes.head()

,title,rating,description,image_dump,tv_program_id
0,Proof,ML,Award-winning journalist Guyon Espiner looks i...,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQEASAB...",0


In [ ]:
df_episodes.to_csv('../data/episodes.csv', index=False)